In [1]:
import h5py
import numpy as np
import os.path
import plotly.graph_objects as go
import plotly.io as pio
import os
from PIL import Image
import imageio.v2 as imageio
pio.renderers.default = 'browser'

## Read HDF5 file

In [2]:
f = h5py.File("fileout.h5", "r")
for key in f.keys():
    print(key, ":", type(f[key])) #Names of the root level object names in HDF5 file - can be groups or datasets.

B0x : <class 'h5py._hl.dataset.Dataset'>
B0y : <class 'h5py._hl.dataset.Dataset'>
B0z : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint112 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint128 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint144 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint16 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint160 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint176 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint192 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint208 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint224 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint240 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint256 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint272 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint288 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint304 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint32 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint320 : <class 'h5py._hl.dataset.Dataset'>
E_abs__tint336 : <class 'h5py._hl.dataset.Datas

In [3]:
for key in f["detAnt_01"].keys():
    print(key, ":",type(f["detAnt_01"][key])) #Names of the root level object names in HDF5 file - can be groups or datasets.

detAnt_ypos : <class 'h5py._hl.dataset.Dataset'>
detAnt_zpos : <class 'h5py._hl.dataset.Dataset'>
i : <class 'h5py._hl.dataset.Dataset'>
rms_E : <class 'h5py._hl.dataset.Dataset'>
sum_EE : <class 'h5py._hl.dataset.Dataset'>
sum_ExEx : <class 'h5py._hl.dataset.Dataset'>
sum_EyEy : <class 'h5py._hl.dataset.Dataset'>
sum_EzEz : <class 'h5py._hl.dataset.Dataset'>


## Plot data HDF5(2D): Electric field

'aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
             'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
             'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
             'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
             'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
             'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd'

In [17]:
x_slice = 100
output_gif = "simulation.gif"
images = []

for i in range(1,51):
    E_field_ds = f[f'E_abs__tint{16*i}'][:]
    sliced_data = E_field_ds[x_slice, :, :]

    fig = go.Figure( data=go.Heatmap(
        z = sliced_data,     # Flattened volume data
        colorscale='jet',                 # Choose a colorscale
        zmin=0,
        zmax=2
    ))
    
    # Update layout
    fig.update_layout(
        title=f"Abs(E) = {i}",
        xaxis_title="Z",
        yaxis_title="Y",
    )
    
    # Show the plot
    #pio.write_image(fig, f"{folder_path}/E_field(t)={i}", format="png")
    img_bytes = fig.to_image(format="png",width=1600,height=1200)
    img = imageio.imread(img_bytes)
    images.append(img)
    #fig.show()

imageio.mimsave(f'{output_gif}', images, duration=200)

## Plot data HDF5: Electric Field 3D

In [4]:
density = f['E_abs__tint00064'][:]
density = np.where(density < 0.01, 0, density)

In [11]:
print(f"Shape: {density.shape}")  # Shape of the dataset (dimensions)
print(f"Data Type (dtype): {density.dtype}")  # Data type of the elements
print(f"Size: {density.size}")  # Total number of elements in the dataset

Shape: (200, 150, 100)
Data Type (dtype): float64
Size: 3000000


In [12]:
# Generate the coordinate grid for the 3D volume (X, Y, Z)
x = density.shape[0]
y = density.shape[1]
z = density.shape[2]
X, Y, Z = np.mgrid[ 0:x:x*1j, 0:y:y*1j, 0:z:z*1j ]

In [13]:
# Create a 3D Volume Plot using Plotly
fig = go.Figure(data=go.Volume(
    x=X.flatten(),                 # X-coordinates
    y=Y.flatten(),                 # Y-coordinates
    z=Z.flatten(),                 # Z-coordinates
    value = density.flatten(),     # Flattened volume data
    isomin=0.01,                    # Minimum value to display
    isomax=np.max(density),        # Maximum value to display
    opacity=0.2,                   # Adjust opacity (between 0 and 1) for better visualization
    surface_count=5,              # Number of volume surfaces to display
    colorscale='plasma',           # Choose a colorscale
))

# Update layout
fig.update_layout(scene=dict(
    xaxis_title='X Axis',
    yaxis_title='Y Axis',
    zaxis_title='Z Axis'
))

# Show the plot
pio.write_image(fig, "", format="png")
fig.show()

In [4]:
E_field_ds = f['E_abs__tint64'][:]
print(f"Shape: {E_field_ds.shape}")  # Shape of the dataset (dimensions)
print(f"Data Type (dtype): {E_field_ds.dtype}")  # Data type of the elements
print(f"Size: {E_field_ds.size}")  # Total number of elements in the dataset

Shape: (200, 150, 100)
Data Type (dtype): float64
Size: 3000000


## Magnetic field

In [16]:
B_field_ds = E_f['B_abs'][:]
print(f"Shape: {B_field_ds.shape}")  # Shape of the dataset (dimensions)
print(f"Data Type (dtype): {B_field_ds.dtype}")  # Data type of the elements
print(f"Size: {B_field_ds.size}")  # Total number of elements in the dataset

Shape: (150, 150, 200)
Data Type (dtype): float64
Size: 4500000


In [17]:
# Create a 3D Volume Plot using Plotly
fig = go.Figure(data=go.Volume(
    x=X.flatten(),                 # X-coordinates
    y=Y.flatten(),                 # Y-coordinates
    z=Z.flatten(),                 # Z-coordinates
    value = B_field_ds.flatten(),     # Flattened volume data
    isomin=0.2,                    # Minimum value to display
    isomax=np.max(B_field_ds),        # Maximum value to display
    opacity=0.3,                   # Adjust opacity (between 0 and 1) for better visualization
    surface_count=10,              # Number of volume surfaces to display
    colorscale='plasma',           # Choose a colorscale
))

# Update layout
fig.update_layout(scene=dict(
    xaxis_title='X Axis',
    yaxis_title='Y Axis',
    zaxis_title='Z Axis'
))

# Show the plot
fig.show()